# Results in the test set

## Import needed packages

In [1]:
import SimpleITK as sitk
from os import listdir
import numpy as np
import re
import scipy
import h5py
import math
from numpy import save
from numpy import savez_compressed
from numpy import load
from volumentations import *
from random import randrange
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import initializers
from tensorflow.keras import backend as K
import gc
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from sklearn.metrics import fbeta_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy import stats
from scipy import ndimage
from patchify import patchify, unpatchify
import segmentation_models_3D as sm
import matplotlib.gridspec as gridspec
import nibabel as nb
from scipy.stats import kruskal
from scipy.stats import false_discovery_control

2025-09-08 19:42:34.047134: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-08 19:42:34.067395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-08 19:42:34.088944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-08 19:42:34.095323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-08 19:42:34.112428: I tensorflow/core/platform/cpu_feature_guar

Segmentation Models: using `keras` framework.


## Ingest data in the training set

In [2]:
# Paths to the train directory with original MRIs and segmentations
MRIs_train_dir = '/home/ch134560/data/Train/'

# Create list with all the MRI files
MRI_train_files = [MRIs_train_dir + MRI_train_file for MRI_train_file in listdir(MRIs_train_dir)]
MRI_train_files.sort()


## Iterate through the MRIs 
# Initiliaze values
current_case = re.search('case(\d+)', MRI_train_files[0])
if current_case: 
    current_case = current_case.group()
else:
    current_case = 'case missing'
current_scan = re.search('s(\d{2})', MRI_train_files[0])
if current_scan: 
    current_scan = current_scan.group()
else:
    current_scan = 'scan missing'
T1 = 0
T2 = 0
flair = 0
segmentation = 0
case_files = [None, None, None, None]
included_train_files = []

# Iterate through the MRIs 
for train_MRI in MRI_train_files:   
    # Define the case
    case = re.search('case(\d+)', train_MRI)
    if case: 
        case = case.group()
    else:
        case = 'case missing'
    scan = re.search('s(\d{2})', train_MRI)
    if scan:
        scan = scan.group()
    else:
        scan = 'scan missing'
    # Update the current case if case is new
    if case != current_case:
        current_case = case
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]
    if scan != current_scan:
        current_scan = scan
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0     
        case_files = [None, None, None, None]
        
    # Include only cases with a T1, T2, FLAIR, and a segmentation
    if ('FLAIR' not in train_MRI) and ('lesion' not in train_MRI) and ('t2w' not in train_MRI) and T1==0:
        case_files[0] = train_MRI
        T1 += 1    
    if ('t2w' in train_MRI) and T2==0:
        case_files[1] = train_MRI
        T2 += 1    
    if ('FLAIR' in train_MRI) and flair==0:
        case_files[2] = train_MRI
        flair += 1
    if ('lesion' in train_MRI) and segmentation==0:
        case_files[3] = train_MRI
        segmentation += 1
    if T1 == 1 and T2 == 1 and flair == 1 and segmentation == 1 and len(case_files) == 4:
        included_train_files = included_train_files + [tuple(case_files)]
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_558537/1097486306.py:11: SyntaxWarning: invalid escape sequence '\d'
  current_case = re.search('case(\d+)', MRI_train_files[0])
/tmp/ipykernel_558537/1097486306.py:16: SyntaxWarning: invalid escape sequence '\d'
  current_scan = re.search('s(\d{2})', MRI_train_files[0])
/tmp/ipykernel_558537/1097486306.py:31: SyntaxWarning: invalid escape sequence '\d'
  case = re.search('case(\d+)', train_MRI)
/tmp/ipykernel_558537/1097486306.py:36: SyntaxWarning: invalid escape sequence '\d'
  scan = re.search('s(\d{2})', train_MRI)


In [3]:
# Initialize the X_train and Y_train tensors
n_all_train = len(included_train_files) * 64
n_channel_train = 3
n_class_train = 2

# Initial images are 256x256x256, but they are too big for the CNN, so they are patchified into 64x64x64 patches
SX_img, SY_img, SZ_img= 256, 256, 256
SX, SY, SZ= 64, 64, 64
X_train = np.zeros( (n_all_train, SX, SY, SZ, n_channel_train), np.float32)
Y_train = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

In [4]:
# Iterate over the dataset to create the entire X_train and Y_train tensors
i = 0

for MRI_T1_file, MRI_T2_file, MRI_FLAIR_file, segmentation_file in included_train_files:

    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T1_img = sitk.ReadImage(MRI_T1_file)
    MRI_T1_img = sitk.GetArrayFromImage(MRI_T1_img)

    # Normalize the image between 0 and 1:
    MRI_T1_img_max = np.max(MRI_T1_img)
    MRI_T1_img_min = np.min(MRI_T1_img)
    MRI_T1_img = (np.float32(MRI_T1_img) - np.float32(MRI_T1_img_min)) / (np.float32(MRI_T1_img_max) - np.float32(MRI_T1_img_min))
        
    # Resize for consistency
    MRI_T1_img = np.float32(np.transpose(MRI_T1_img, [2, 1, 0]))
    MRI_T1_img = scipy.ndimage.zoom(MRI_T1_img, zoom=(SX_img/MRI_T1_img.shape[0], SY_img/MRI_T1_img.shape[1], SZ_img/MRI_T1_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T1_img_patches = patchify(MRI_T1_img, (64, 64, 64), step=64)
    MRI_T1_img_patches = np.reshape(MRI_T1_img_patches, (-1, MRI_T1_img_patches.shape[3], MRI_T1_img_patches.shape[4], MRI_T1_img_patches.shape[5]))
    X_train[i:i+64, :, :, :, 0] = MRI_T1_img_patches 


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T2_img = sitk.ReadImage(MRI_T2_file)
    MRI_T2_img = sitk.GetArrayFromImage(MRI_T2_img)

    # Normalize the image between 0 and 1:
    MRI_T2_img_max = np.max(MRI_T2_img)
    MRI_T2_img_min = np.min(MRI_T2_img)
    MRI_T2_img = (np.float32(MRI_T2_img) - np.float32(MRI_T2_img_min)) / (np.float32(MRI_T2_img_max) - np.float32(MRI_T2_img_min))
        
    # Resize for consistency
    MRI_T2_img = np.float32(np.transpose(MRI_T2_img, [2, 1, 0]))
    MRI_T2_img = scipy.ndimage.zoom(MRI_T2_img, zoom=(SX_img/MRI_T2_img.shape[0], SY_img/MRI_T2_img.shape[1], SZ_img/MRI_T2_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T2_img_patches = patchify(MRI_T2_img, (64, 64, 64), step=64)
    MRI_T2_img_patches = np.reshape(MRI_T2_img_patches, (-1, MRI_T2_img_patches.shape[3], MRI_T2_img_patches.shape[4], MRI_T2_img_patches.shape[5]))
    X_train[i:i+64, :, :, :, 1] = MRI_T2_img_patches 

    
    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_FLAIR_img = sitk.ReadImage(MRI_FLAIR_file)
    MRI_FLAIR_img = sitk.GetArrayFromImage(MRI_FLAIR_img)

    # Normalize the image between 0 and 1:
    MRI_FLAIR_img_max = np.max(MRI_FLAIR_img)
    MRI_FLAIR_img_min = np.min(MRI_FLAIR_img)
    MRI_FLAIR_img = (np.float32(MRI_FLAIR_img) - np.float32(MRI_FLAIR_img_min)) / (np.float32(MRI_FLAIR_img_max) - np.float32(MRI_FLAIR_img_min))
        
    # Resize for consistency
    MRI_FLAIR_img = np.float32(np.transpose(MRI_FLAIR_img, [2, 1, 0]))
    MRI_FLAIR_img = scipy.ndimage.zoom(MRI_FLAIR_img, zoom=(SX_img/MRI_FLAIR_img.shape[0], SY_img/MRI_FLAIR_img.shape[1], SZ_img/MRI_FLAIR_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_FLAIR_img_patches = patchify(MRI_FLAIR_img, (64, 64, 64), step=64)
    MRI_FLAIR_img_patches = np.reshape(MRI_FLAIR_img_patches, (-1, MRI_FLAIR_img_patches.shape[3], MRI_FLAIR_img_patches.shape[4], MRI_FLAIR_img_patches.shape[5]))
    X_train[i:i+64, :, :, :, 2] = MRI_FLAIR_img_patches        
        


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    segmentation_img = sitk.ReadImage(segmentation_file)
    segmentation_img = sitk.GetArrayFromImage(segmentation_img)
    
    # Do not normalize because segmentations should be 1s and 0s 

    # Resize for consistency
    segmentation_img = np.transpose(segmentation_img, [2, 1, 0])
    segmentation_img = scipy.ndimage.zoom(segmentation_img, zoom=(SX_img/segmentation_img.shape[0], SY_img/segmentation_img.shape[1], SZ_img/segmentation_img.shape[2]), order=0)

    # Patchify with a homemade method and introduce the MRI into the Y tensor
    segmentation_img_patches = patchify(segmentation_img, (64, 64, 64), step=64)
    segmentation_img_patches = np.reshape(segmentation_img_patches, (-1, segmentation_img_patches.shape[3], segmentation_img_patches.shape[4], segmentation_img_patches.shape[5]))
    Y_train[i:i+64, :, :, :, 0] = segmentation_img_patches     
    
    # Update counting value
    i += 64

In [5]:
X_train.shape, Y_train.shape

((11264, 64, 64, 64, 3), (11264, 64, 64, 64, 1))

In [6]:
# Assumed that 2s and 3s are not markers of tubers
Y_train[Y_train > 1] = 0

In [7]:
# Change the number type to float 32
Y_train = Y_train.astype(np.float32)

## Ingest data in the validation set

In [8]:
# Paths to the validation directory with original MRIs and segmentations
MRIs_validation_dir = '/home/ch134560/data/Validation/'

# Create list with all the MRI files
MRI_validation_files = [MRIs_validation_dir + MRI_validation_file for MRI_validation_file in listdir(MRIs_validation_dir)]
MRI_validation_files.sort()


## Iterate through the MRIs 
# Initiliaze values
current_case = re.search('case(\d+)', MRI_validation_files[0])
if current_case: 
    current_case = current_case.group()
else:
    current_case = 'case missing'
current_scan = re.search('s(\d{2})', MRI_validation_files[0])
if current_scan: 
    current_scan = current_scan.group()
else:
    current_scan = 'scan missing'
T1 = 0
T2 = 0
flair = 0
segmentation = 0
case_files = [None, None, None, None]
included_validation_files = []

# Iterate through the MRIs 
for validation_MRI in MRI_validation_files:   
    # Define the case
    case = re.search('case(\d+)', validation_MRI)
    if case: 
        case = case.group()
    else:
        case = 'case missing'
    scan = re.search('s(\d{2})', validation_MRI)
    if scan:
        scan = scan.group()
    else:
        scan = 'scan missing'
    # Update the current case if case is new
    if case != current_case:
        current_case = case
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]
    if scan != current_scan:
        current_scan = scan
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0     
        case_files = [None, None, None, None]
        
    # Include only cases with a T1, T2, FLAIR, and a segmentation
    if ('FLAIR' not in validation_MRI) and ('lesion' not in validation_MRI) and ('t2w' not in validation_MRI) and T1==0:
        case_files[0] = validation_MRI
        T1 += 1    
    if ('t2w' in validation_MRI) and T2==0:
        case_files[1] = validation_MRI
        T2 += 1    
    if ('FLAIR' in validation_MRI) and flair==0:
        case_files[2] = validation_MRI
        flair += 1
    if ('lesion' in validation_MRI) and segmentation==0:
        case_files[3] = validation_MRI
        segmentation += 1
    if T1 == 1 and T2 == 1 and flair == 1 and segmentation == 1 and len(case_files) == 4:
        included_validation_files = included_validation_files + [tuple(case_files)]
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_558537/2662715101.py:11: SyntaxWarning: invalid escape sequence '\d'
  current_case = re.search('case(\d+)', MRI_validation_files[0])
/tmp/ipykernel_558537/2662715101.py:16: SyntaxWarning: invalid escape sequence '\d'
  current_scan = re.search('s(\d{2})', MRI_validation_files[0])
/tmp/ipykernel_558537/2662715101.py:31: SyntaxWarning: invalid escape sequence '\d'
  case = re.search('case(\d+)', validation_MRI)
/tmp/ipykernel_558537/2662715101.py:36: SyntaxWarning: invalid escape sequence '\d'
  scan = re.search('s(\d{2})', validation_MRI)


In [9]:
# Initialize the X_validation and Y_validation tensors
n_all_validation = len(included_validation_files) * 64
n_channel_validation = 3
n_class_validation = 2

# Initial images are 256x256x256, but they are too big for the CNN, so they are patchified into 64x64x64 patches
SX_img, SY_img, SZ_img= 256, 256, 256
SX, SY, SZ= 64, 64, 64
X_validation = np.zeros( (n_all_validation, SX, SY, SZ, n_channel_validation), np.float32)
Y_validation = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

In [10]:
# Iterate over the dataset to create the entire X_validation and Y_validation tensors
i = 0

for MRI_T1_file, MRI_T2_file, MRI_FLAIR_file, segmentation_file in included_validation_files:

    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T1_img = sitk.ReadImage(MRI_T1_file)
    MRI_T1_img = sitk.GetArrayFromImage(MRI_T1_img)

    # Normalize the image between 0 and 1:
    MRI_T1_img_max = np.max(MRI_T1_img)
    MRI_T1_img_min = np.min(MRI_T1_img)
    MRI_T1_img = (np.float32(MRI_T1_img) - np.float32(MRI_T1_img_min)) / (np.float32(MRI_T1_img_max) - np.float32(MRI_T1_img_min))
        
    # Resize for consistency
    MRI_T1_img = np.float32(np.transpose(MRI_T1_img, [2, 1, 0]))
    MRI_T1_img = scipy.ndimage.zoom(MRI_T1_img, zoom=(SX_img/MRI_T1_img.shape[0], SY_img/MRI_T1_img.shape[1], SZ_img/MRI_T1_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T1_img_patches = patchify(MRI_T1_img, (64, 64, 64), step=64)
    MRI_T1_img_patches = np.reshape(MRI_T1_img_patches, (-1, MRI_T1_img_patches.shape[3], MRI_T1_img_patches.shape[4], MRI_T1_img_patches.shape[5]))
    X_validation[i:i+64, :, :, :, 0] = MRI_T1_img_patches 


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T2_img = sitk.ReadImage(MRI_T2_file)
    MRI_T2_img = sitk.GetArrayFromImage(MRI_T2_img)

    # Normalize the image between 0 and 1:
    MRI_T2_img_max = np.max(MRI_T2_img)
    MRI_T2_img_min = np.min(MRI_T2_img)
    MRI_T2_img = (np.float32(MRI_T2_img) - np.float32(MRI_T2_img_min)) / (np.float32(MRI_T2_img_max) - np.float32(MRI_T2_img_min))
        
    # Resize for consistency
    MRI_T2_img = np.float32(np.transpose(MRI_T2_img, [2, 1, 0]))
    MRI_T2_img = scipy.ndimage.zoom(MRI_T2_img, zoom=(SX_img/MRI_T2_img.shape[0], SY_img/MRI_T2_img.shape[1], SZ_img/MRI_T2_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T2_img_patches = patchify(MRI_T2_img, (64, 64, 64), step=64)
    MRI_T2_img_patches = np.reshape(MRI_T2_img_patches, (-1, MRI_T2_img_patches.shape[3], MRI_T2_img_patches.shape[4], MRI_T2_img_patches.shape[5]))
    X_validation[i:i+64, :, :, :, 1] = MRI_T2_img_patches 

    
    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_FLAIR_img = sitk.ReadImage(MRI_FLAIR_file)
    MRI_FLAIR_img = sitk.GetArrayFromImage(MRI_FLAIR_img)

    # Normalize the image between 0 and 1:
    MRI_FLAIR_img_max = np.max(MRI_FLAIR_img)
    MRI_FLAIR_img_min = np.min(MRI_FLAIR_img)
    MRI_FLAIR_img = (np.float32(MRI_FLAIR_img) - np.float32(MRI_FLAIR_img_min)) / (np.float32(MRI_FLAIR_img_max) - np.float32(MRI_FLAIR_img_min))
        
    # Resize for consistency
    MRI_FLAIR_img = np.float32(np.transpose(MRI_FLAIR_img, [2, 1, 0]))
    MRI_FLAIR_img = scipy.ndimage.zoom(MRI_FLAIR_img, zoom=(SX_img/MRI_FLAIR_img.shape[0], SY_img/MRI_FLAIR_img.shape[1], SZ_img/MRI_FLAIR_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_FLAIR_img_patches = patchify(MRI_FLAIR_img, (64, 64, 64), step=64)
    MRI_FLAIR_img_patches = np.reshape(MRI_FLAIR_img_patches, (-1, MRI_FLAIR_img_patches.shape[3], MRI_FLAIR_img_patches.shape[4], MRI_FLAIR_img_patches.shape[5]))
    X_validation[i:i+64, :, :, :, 2] = MRI_FLAIR_img_patches        
        


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    segmentation_img = sitk.ReadImage(segmentation_file)
    segmentation_img = sitk.GetArrayFromImage(segmentation_img)
    
    # Do not normalize because segmentations should be 1s and 0s 

    # Resize for consistency
    segmentation_img = np.transpose(segmentation_img, [2, 1, 0])
    segmentation_img = scipy.ndimage.zoom(segmentation_img, zoom=(SX_img/segmentation_img.shape[0], SY_img/segmentation_img.shape[1], SZ_img/segmentation_img.shape[2]), order=0)

    # Patchify with a homemade method and introduce the MRI into the Y tensor
    segmentation_img_patches = patchify(segmentation_img, (64, 64, 64), step=64)
    segmentation_img_patches = np.reshape(segmentation_img_patches, (-1, segmentation_img_patches.shape[3], segmentation_img_patches.shape[4], segmentation_img_patches.shape[5]))
    Y_validation[i:i+64, :, :, :, 0] = segmentation_img_patches     
    
    # Update counting value
    i += 64

In [11]:
X_validation.shape, Y_validation.shape

((2496, 64, 64, 64, 3), (2496, 64, 64, 64, 1))

In [12]:
# Assumed that 2s and 3s are not markers of tubers
Y_validation[Y_validation > 1] = 0

In [13]:
# Change the number type to float 32
Y_validation = Y_validation.astype(np.float32)

## Ingest data in the test set

In [14]:
# Paths to the test directory with original MRIs and segmentations
MRIs_test_dir = '/home/ch134560/data/Test/'

# Create list with all the MRI files
MRI_test_files = [MRIs_test_dir + MRI_test_file for MRI_test_file in listdir(MRIs_test_dir)]
MRI_test_files.sort()


## Iterate through the MRIs 
# Initiliaze values
current_case = re.search('case(\d+)', MRI_test_files[0])
if current_case: 
    current_case = current_case.group()
else:
    current_case = 'case missing'
current_scan = re.search('s(\d{2})', MRI_test_files[0])
if current_scan: 
    current_scan = current_scan.group()
else:
    current_scan = 'scan missing'
T1 = 0
T2 = 0
flair = 0
segmentation = 0
case_files = [None, None, None, None]
included_test_files = []

# Iterate through the MRIs 
for test_MRI in MRI_test_files:   
    # Define the case
    case = re.search('case(\d+)', test_MRI)
    if case: 
        case = case.group()
    else:
        case = 'case missing'
    scan = re.search('s(\d{2})', test_MRI)
    if scan:
        scan = scan.group()
    else:
        scan = 'scan missing'
    # Update the current case if case is new
    if case != current_case:
        current_case = case
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]
    if scan != current_scan:
        current_scan = scan
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0     
        case_files = [None, None, None, None]
        
    # Include only cases with a T1, T2, FLAIR, and a segmentation
    if ('FLAIR' not in test_MRI) and ('lesion' not in test_MRI) and ('t2w' not in test_MRI) and T1==0:
        case_files[0] = test_MRI
        T1 += 1    
    if ('t2w' in test_MRI) and T2==0:
        case_files[1] = test_MRI
        T2 += 1    
    if ('FLAIR' in test_MRI) and flair==0:
        case_files[2] = test_MRI
        flair += 1
    if ('lesion' in test_MRI) and segmentation==0:
        case_files[3] = test_MRI
        segmentation += 1
    if T1 == 1 and T2 == 1 and flair == 1 and segmentation == 1 and len(case_files) == 4:
        included_test_files = included_test_files + [tuple(case_files)]
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_558537/1046949074.py:11: SyntaxWarning: invalid escape sequence '\d'
  current_case = re.search('case(\d+)', MRI_test_files[0])
/tmp/ipykernel_558537/1046949074.py:16: SyntaxWarning: invalid escape sequence '\d'
  current_scan = re.search('s(\d{2})', MRI_test_files[0])
/tmp/ipykernel_558537/1046949074.py:31: SyntaxWarning: invalid escape sequence '\d'
  case = re.search('case(\d+)', test_MRI)
/tmp/ipykernel_558537/1046949074.py:36: SyntaxWarning: invalid escape sequence '\d'
  scan = re.search('s(\d{2})', test_MRI)


In [15]:
# Initialize the X_test and Y_test tensors
n_all_test = len(included_test_files) * 64
n_channel_test = 3
n_class_test = 2

# Initial images are 256x256x256, but they are too big for the CNN, so they are patchified into 64x64x64 patches
SX_img, SY_img, SZ_img= 256, 256, 256
SX, SY, SZ= 64, 64, 64
X_test = np.zeros( (n_all_test, SX, SY, SZ, n_channel_test), np.float32)
Y_test = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

In [16]:
# Iterate over the dataset to create the entire X_test and Y_test tensors
i = 0

for MRI_T1_file, MRI_T2_file, MRI_FLAIR_file, segmentation_file in included_test_files:

    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T1_img = sitk.ReadImage(MRI_T1_file)
    MRI_T1_img = sitk.GetArrayFromImage(MRI_T1_img)

    # Normalize the image between 0 and 1:
    MRI_T1_img_max = np.max(MRI_T1_img)
    MRI_T1_img_min = np.min(MRI_T1_img)
    MRI_T1_img = (np.float32(MRI_T1_img) - np.float32(MRI_T1_img_min)) / (np.float32(MRI_T1_img_max) - np.float32(MRI_T1_img_min))
        
    # Resize for consistency
    MRI_T1_img = np.float32(np.transpose(MRI_T1_img, [2, 1, 0]))
    MRI_T1_img = scipy.ndimage.zoom(MRI_T1_img, zoom=(SX_img/MRI_T1_img.shape[0], SY_img/MRI_T1_img.shape[1], SZ_img/MRI_T1_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T1_img_patches = patchify(MRI_T1_img, (64, 64, 64), step=64)
    MRI_T1_img_patches = np.reshape(MRI_T1_img_patches, (-1, MRI_T1_img_patches.shape[3], MRI_T1_img_patches.shape[4], MRI_T1_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 0] = MRI_T1_img_patches 


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T2_img = sitk.ReadImage(MRI_T2_file)
    MRI_T2_img = sitk.GetArrayFromImage(MRI_T2_img)

    # Normalize the image between 0 and 1:
    MRI_T2_img_max = np.max(MRI_T2_img)
    MRI_T2_img_min = np.min(MRI_T2_img)
    MRI_T2_img = (np.float32(MRI_T2_img) - np.float32(MRI_T2_img_min)) / (np.float32(MRI_T2_img_max) - np.float32(MRI_T2_img_min))
        
    # Resize for consistency
    MRI_T2_img = np.float32(np.transpose(MRI_T2_img, [2, 1, 0]))
    MRI_T2_img = scipy.ndimage.zoom(MRI_T2_img, zoom=(SX_img/MRI_T2_img.shape[0], SY_img/MRI_T2_img.shape[1], SZ_img/MRI_T2_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T2_img_patches = patchify(MRI_T2_img, (64, 64, 64), step=64)
    MRI_T2_img_patches = np.reshape(MRI_T2_img_patches, (-1, MRI_T2_img_patches.shape[3], MRI_T2_img_patches.shape[4], MRI_T2_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 1] = MRI_T2_img_patches 

    
    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_FLAIR_img = sitk.ReadImage(MRI_FLAIR_file)
    MRI_FLAIR_img = sitk.GetArrayFromImage(MRI_FLAIR_img)

    # Normalize the image between 0 and 1:
    MRI_FLAIR_img_max = np.max(MRI_FLAIR_img)
    MRI_FLAIR_img_min = np.min(MRI_FLAIR_img)
    MRI_FLAIR_img = (np.float32(MRI_FLAIR_img) - np.float32(MRI_FLAIR_img_min)) / (np.float32(MRI_FLAIR_img_max) - np.float32(MRI_FLAIR_img_min))
        
    # Resize for consistency
    MRI_FLAIR_img = np.float32(np.transpose(MRI_FLAIR_img, [2, 1, 0]))
    MRI_FLAIR_img = scipy.ndimage.zoom(MRI_FLAIR_img, zoom=(SX_img/MRI_FLAIR_img.shape[0], SY_img/MRI_FLAIR_img.shape[1], SZ_img/MRI_FLAIR_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_FLAIR_img_patches = patchify(MRI_FLAIR_img, (64, 64, 64), step=64)
    MRI_FLAIR_img_patches = np.reshape(MRI_FLAIR_img_patches, (-1, MRI_FLAIR_img_patches.shape[3], MRI_FLAIR_img_patches.shape[4], MRI_FLAIR_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 2] = MRI_FLAIR_img_patches        
        


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    segmentation_img = sitk.ReadImage(segmentation_file)
    segmentation_img = sitk.GetArrayFromImage(segmentation_img)
    
    # Do not normalize because segmentations should be 1s and 0s 

    # Resize for consistency
    segmentation_img = np.transpose(segmentation_img, [2, 1, 0])
    segmentation_img = scipy.ndimage.zoom(segmentation_img, zoom=(SX_img/segmentation_img.shape[0], SY_img/segmentation_img.shape[1], SZ_img/segmentation_img.shape[2]), order=0)

    # Patchify with a homemade method and introduce the MRI into the Y tensor
    segmentation_img_patches = patchify(segmentation_img, (64, 64, 64), step=64)
    segmentation_img_patches = np.reshape(segmentation_img_patches, (-1, segmentation_img_patches.shape[3], segmentation_img_patches.shape[4], segmentation_img_patches.shape[5]))
    Y_test[i:i+64, :, :, :, 0] = segmentation_img_patches     
    
    # Update counting value
    i += 64

In [17]:
X_test.shape, Y_test.shape

((3072, 64, 64, 64, 3), (3072, 64, 64, 64, 1))

In [18]:
# Assumed that 2s and 3s are not markers of tubers
Y_test[Y_test > 1] = 0

In [19]:
# Change the number type to float 32
Y_test = Y_test.astype(np.float32)

## Extract parcellation masks and calculate Dice coefficient (total and in lobes)

In [20]:
####################################GRAY MATTER MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
gray_matter_mask = sitk.ReadImage('/home/ch134560/data/masks/gray_matter_total_lobes_binarized.nii')
gray_matter_mask = sitk.GetArrayFromImage(gray_matter_mask)

# Resize for consistency
gray_matter_mask = np.float32(np.transpose(gray_matter_mask, [2, 1, 0]))
gray_matter_mask = scipy.ndimage.zoom(gray_matter_mask, zoom=(SX_img/gray_matter_mask.shape[0], SY_img/gray_matter_mask.shape[1], SZ_img/gray_matter_mask.shape[2]), order=0)




####################################RIGHT OCCIPITAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_occipital_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Occipital_Lobe_mask.nii.gz')
R_occipital_mask = sitk.GetArrayFromImage(R_occipital_mask)

# Resize for consistency
R_occipital_mask = np.float32(np.transpose(R_occipital_mask, [2, 1, 0]))
R_occipital_mask = scipy.ndimage.zoom(R_occipital_mask, zoom=(SX_img/R_occipital_mask.shape[0], SY_img/R_occipital_mask.shape[1], SZ_img/R_occipital_mask.shape[2]), order=0)




####################################LEFT OCCIPITAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_occipital_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Occipital_Lobe_mask.nii.gz')
L_occipital_mask = sitk.GetArrayFromImage(L_occipital_mask)

# Resize for consistency
L_occipital_mask = np.float32(np.transpose(L_occipital_mask, [2, 1, 0]))
L_occipital_mask = scipy.ndimage.zoom(L_occipital_mask, zoom=(SX_img/L_occipital_mask.shape[0], SY_img/L_occipital_mask.shape[1], SZ_img/L_occipital_mask.shape[2]), order=0)




####################################RIGHT PARIETAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_parietal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Parietal_Lobe_mask.nii.gz')
R_parietal_mask = sitk.GetArrayFromImage(R_parietal_mask)

# Resize for consistency
R_parietal_mask = np.float32(np.transpose(R_parietal_mask, [2, 1, 0]))
R_parietal_mask = scipy.ndimage.zoom(R_parietal_mask, zoom=(SX_img/R_parietal_mask.shape[0], SY_img/R_parietal_mask.shape[1], SZ_img/R_parietal_mask.shape[2]), order=0)




####################################LEFT PARIETAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_parietal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Parietal_Lobe_mask.nii.gz')
L_parietal_mask = sitk.GetArrayFromImage(L_parietal_mask)

# Resize for consistency
L_parietal_mask = np.float32(np.transpose(L_parietal_mask, [2, 1, 0]))
L_parietal_mask = scipy.ndimage.zoom(L_parietal_mask, zoom=(SX_img/L_parietal_mask.shape[0], SY_img/L_parietal_mask.shape[1], SZ_img/L_parietal_mask.shape[2]), order=0)




####################################RIGHT TEMPORAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_temporal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Temporal_Lobe_mask.nii')
R_temporal_mask = sitk.GetArrayFromImage(R_temporal_mask)

# Resize for consistency
R_temporal_mask = np.float32(np.transpose(R_temporal_mask, [2, 1, 0]))
R_temporal_mask = scipy.ndimage.zoom(R_temporal_mask, zoom=(SX_img/R_temporal_mask.shape[0], SY_img/R_temporal_mask.shape[1], SZ_img/R_temporal_mask.shape[2]), order=0)




####################################LEFT TEMPORAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_temporal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Temporal_Lobe_mask.nii')
L_temporal_mask = sitk.GetArrayFromImage(L_temporal_mask)

# Resize for consistency
L_temporal_mask = np.float32(np.transpose(L_temporal_mask, [2, 1, 0]))
L_temporal_mask = scipy.ndimage.zoom(L_temporal_mask, zoom=(SX_img/L_temporal_mask.shape[0], SY_img/L_temporal_mask.shape[1], SZ_img/L_temporal_mask.shape[2]), order=0)




####################################RIGHT FRONTAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_frontal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Frontal_Lobe_mask.nii.gz')
R_frontal_mask = sitk.GetArrayFromImage(R_frontal_mask)

# Resize for consistency
R_frontal_mask = np.float32(np.transpose(R_frontal_mask, [2, 1, 0]))
R_frontal_mask = scipy.ndimage.zoom(R_frontal_mask, zoom=(SX_img/R_frontal_mask.shape[0], SY_img/R_frontal_mask.shape[1], SZ_img/R_frontal_mask.shape[2]), order=0)




####################################LEFT FRONTAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_frontal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Frontal_Lobe_mask.nii.gz')
L_frontal_mask = sitk.GetArrayFromImage(L_frontal_mask)

# Resize for consistency
L_frontal_mask = np.float32(np.transpose(L_frontal_mask, [2, 1, 0]))
L_frontal_mask = scipy.ndimage.zoom(L_frontal_mask, zoom=(SX_img/L_frontal_mask.shape[0], SY_img/L_frontal_mask.shape[1], SZ_img/L_frontal_mask.shape[2]), order=0)

## Train set

#### Mask: gray matter

In [21]:
# Initialize the tensor where the reformatted masks will be saved
Y_gray_matter_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    gray_matter_mask_patches = patchify(gray_matter_mask, (64, 64, 64), step=64)
    gray_matter_mask_patches = np.reshape(gray_matter_mask_patches, (-1, gray_matter_mask_patches.shape[3], gray_matter_mask_patches.shape[4], gray_matter_mask_patches.shape[5]))
    Y_gray_matter_mask[i*64:i*64+64, :, :, :, 0] = gray_matter_mask_patches    

Y_gray_matter_mask = Y_gray_matter_mask.astype(np.float32)

In [22]:
# Mask the Y_train
Y_train_gray_matter = Y_train * Y_gray_matter_mask

#### Mask: left frontal

In [23]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_frontal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_frontal_mask_patches = patchify(L_frontal_mask, (64, 64, 64), step=64)
    L_frontal_mask_patches = np.reshape(L_frontal_mask_patches, (-1, L_frontal_mask_patches.shape[3], L_frontal_mask_patches.shape[4], L_frontal_mask_patches.shape[5]))
    Y_L_frontal_mask[i*64:i*64+64, :, :, :, 0] = L_frontal_mask_patches    

Y_L_frontal_mask = Y_L_frontal_mask.astype(np.float32)

In [24]:
# Mask the Y_train and Y_predictions
Y_train_L_frontal = Y_train * Y_L_frontal_mask

#### Mask: right frontal

In [25]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_frontal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_frontal_mask_patches = patchify(R_frontal_mask, (64, 64, 64), step=64)
    R_frontal_mask_patches = np.reshape(R_frontal_mask_patches, (-1, R_frontal_mask_patches.shape[3], R_frontal_mask_patches.shape[4], R_frontal_mask_patches.shape[5]))
    Y_R_frontal_mask[i*64:i*64+64, :, :, :, 0] = R_frontal_mask_patches    

Y_R_frontal_mask = Y_R_frontal_mask.astype(np.float32)

In [26]:
# Mask the Y_train and Y_predictions
Y_train_R_frontal = Y_train * Y_R_frontal_mask

#### Mask: left temporal

In [27]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_temporal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_temporal_mask_patches = patchify(L_temporal_mask, (64, 64, 64), step=64)
    L_temporal_mask_patches = np.reshape(L_temporal_mask_patches, (-1, L_temporal_mask_patches.shape[3], L_temporal_mask_patches.shape[4], L_temporal_mask_patches.shape[5]))
    Y_L_temporal_mask[i*64:i*64+64, :, :, :, 0] = L_temporal_mask_patches    

Y_L_temporal_mask = Y_L_temporal_mask.astype(np.float32)

In [28]:
# Mask the Y_train and Y_predictions
Y_train_L_temporal = Y_train * Y_L_temporal_mask

#### Mask: right temporal

In [29]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_temporal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_temporal_mask_patches = patchify(R_temporal_mask, (64, 64, 64), step=64)
    R_temporal_mask_patches = np.reshape(R_temporal_mask_patches, (-1, R_temporal_mask_patches.shape[3], R_temporal_mask_patches.shape[4], R_temporal_mask_patches.shape[5]))
    Y_R_temporal_mask[i*64:i*64+64, :, :, :, 0] = R_temporal_mask_patches    

Y_R_temporal_mask = Y_R_temporal_mask.astype(np.float32)

In [30]:
# Mask the Y_train and Y_predictions
Y_train_R_temporal = Y_train * Y_R_temporal_mask

#### Mask: left parietal

In [31]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_parietal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_parietal_mask_patches = patchify(L_parietal_mask, (64, 64, 64), step=64)
    L_parietal_mask_patches = np.reshape(L_parietal_mask_patches, (-1, L_parietal_mask_patches.shape[3], L_parietal_mask_patches.shape[4], L_parietal_mask_patches.shape[5]))
    Y_L_parietal_mask[i*64:i*64+64, :, :, :, 0] = L_parietal_mask_patches    

Y_L_parietal_mask = Y_L_parietal_mask.astype(np.float32)

In [32]:
# Mask the Y_train and Y_predictions
Y_train_L_parietal = Y_train * Y_L_parietal_mask

#### Mask: right parietal

In [33]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_parietal_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_parietal_mask_patches = patchify(R_parietal_mask, (64, 64, 64), step=64)
    R_parietal_mask_patches = np.reshape(R_parietal_mask_patches, (-1, R_parietal_mask_patches.shape[3], R_parietal_mask_patches.shape[4], R_parietal_mask_patches.shape[5]))
    Y_R_parietal_mask[i*64:i*64+64, :, :, :, 0] = R_parietal_mask_patches    

Y_R_parietal_mask = Y_R_parietal_mask.astype(np.float32)

In [34]:
# Mask the Y_train and Y_predictions
Y_train_R_parietal = Y_train * Y_R_parietal_mask

#### Mask: left occipital

In [35]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_occipital_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_occipital_mask_patches = patchify(L_occipital_mask, (64, 64, 64), step=64)
    L_occipital_mask_patches = np.reshape(L_occipital_mask_patches, (-1, L_occipital_mask_patches.shape[3], L_occipital_mask_patches.shape[4], L_occipital_mask_patches.shape[5]))
    Y_L_occipital_mask[i*64:i*64+64, :, :, :, 0] = L_occipital_mask_patches    

Y_L_occipital_mask = Y_L_occipital_mask.astype(np.float32)

In [36]:
# Mask the Y_train and Y_predictions
Y_train_L_occipital = Y_train * Y_L_occipital_mask

#### Mask: right occipital

In [37]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_occipital_mask = np.zeros( (n_all_train, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_train and Y_validation tensors
for i in range(len(included_train_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_occipital_mask_patches = patchify(R_occipital_mask, (64, 64, 64), step=64)
    R_occipital_mask_patches = np.reshape(R_occipital_mask_patches, (-1, R_occipital_mask_patches.shape[3], R_occipital_mask_patches.shape[4], R_occipital_mask_patches.shape[5]))
    Y_R_occipital_mask[i*64:i*64+64, :, :, :, 0] = R_occipital_mask_patches    

Y_R_occipital_mask = Y_R_occipital_mask.astype(np.float32)

In [38]:
# Mask the Y_train and Y_predictions
Y_train_R_occipital = Y_train * Y_R_occipital_mask

## Tuber burden per parcellations

#### Whole brain

In [39]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden = []
for i in range(int(Y_train.shape[0]/64)):
    tuber_burden_single_MRI = np.sum(Y_train[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden.append(tuber_burden_single_MRI)
# Save variables
train_whole_brain_tuber_burden = tuber_burden
# Calculate statistics
np.mean(tuber_burden), np.std(tuber_burden), np.median(tuber_burden), np.percentile(tuber_burden, 25), np.percentile(tuber_burden, 75)

(25288.852, 24694.648, 15341.5, 6683.25, 37695.25)

#### Gray matter

In [40]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_gray_matter = []
for i in range(int(Y_train_gray_matter.shape[0]/64)):
    tuber_burden_single_MRI_gray_matter = np.sum(Y_train_gray_matter[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_gray_matter.append(tuber_burden_single_MRI_gray_matter)
# Save variables
train_gray_matter_tuber_burden = tuber_burden_gray_matter
# Calculate statistics
np.mean(tuber_burden_gray_matter), np.std(tuber_burden_gray_matter), np.median(tuber_burden_gray_matter), np.percentile(tuber_burden_gray_matter, 25), np.percentile(tuber_burden_gray_matter, 75)

(23730.25, 23539.408, 14375.5, 5883.5, 35449.5)

#### Left frontal

In [41]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_frontal = []
for i in range(int(Y_train_L_frontal.shape[0]/64)):
    tuber_burden_single_MRI_L_frontal = np.sum(Y_train_L_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_frontal.append(tuber_burden_single_MRI_L_frontal)
# Save variables
train_L_frontal_tuber_burden = tuber_burden_L_frontal
# Calculate statistics
np.mean(tuber_burden_L_frontal), np.std(tuber_burden_L_frontal), np.median(tuber_burden_L_frontal), np.percentile(tuber_burden_L_frontal, 25), np.percentile(tuber_burden_L_frontal, 75)

(5743.352, 6236.711, 3518.5, 1202.5, 8519.5)

#### Right frontal

In [42]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_frontal = []
for i in range(int(Y_train_R_frontal.shape[0]/64)):
    tuber_burden_single_MRI_R_frontal = np.sum(Y_train_R_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_frontal.append(tuber_burden_single_MRI_R_frontal)
# Save variables
train_R_frontal_tuber_burden = tuber_burden_R_frontal
# Calculate statistics
np.mean(tuber_burden_R_frontal), np.std(tuber_burden_R_frontal), np.median(tuber_burden_R_frontal), np.percentile(tuber_burden_R_frontal, 25), np.percentile(tuber_burden_R_frontal, 75)

(5269.5737, 6065.169, 3244.0, 1049.25, 7943.5)

#### Left temporal

In [43]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_temporal = []
for i in range(int(Y_train_L_temporal.shape[0]/64)):
    tuber_burden_single_MRI_L_temporal = np.sum(Y_train_L_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_temporal.append(tuber_burden_single_MRI_L_temporal)
# Save variables
train_L_temporal_tuber_burden = tuber_burden_L_temporal
# Calculate statistics
np.mean(tuber_burden_L_temporal), np.std(tuber_burden_L_temporal), np.median(tuber_burden_L_temporal), np.percentile(tuber_burden_L_temporal, 25), np.percentile(tuber_burden_L_temporal, 75)

(2008.5852, 3486.6184, 630.0, 0.0, 2168.25)

#### Right temporal

In [44]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_temporal = []
for i in range(int(Y_train_R_temporal.shape[0]/64)):
    tuber_burden_single_MRI_R_temporal = np.sum(Y_train_R_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_temporal.append(tuber_burden_single_MRI_R_temporal)
# Save variables
train_R_temporal_tuber_burden = tuber_burden_R_temporal
# Calculate statistics
np.mean(tuber_burden_R_temporal), np.std(tuber_burden_R_temporal), np.median(tuber_burden_R_temporal), np.percentile(tuber_burden_R_temporal, 25), np.percentile(tuber_burden_R_temporal, 75)

(2373.807, 3197.5957, 1184.5, 79.5, 3310.0)

#### Left parietal

In [45]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_parietal = []
for i in range(int(Y_train_L_parietal.shape[0]/64)):
    tuber_burden_single_MRI_L_parietal = np.sum(Y_train_L_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_parietal.append(tuber_burden_single_MRI_L_parietal)
# Save variables
train_L_parietal_tuber_burden = tuber_burden_L_parietal
# Calculate statistics
np.mean(tuber_burden_L_parietal), np.std(tuber_burden_L_parietal), np.median(tuber_burden_L_parietal), np.percentile(tuber_burden_L_parietal, 25), np.percentile(tuber_burden_L_parietal, 75)

(2572.0115, 3156.8938, 1125.0, 191.25, 3959.0)

#### Right parietal

In [46]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_parietal = []
for i in range(int(Y_train_R_parietal.shape[0]/64)):
    tuber_burden_single_MRI_R_parietal = np.sum(Y_train_R_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_parietal.append(tuber_burden_single_MRI_R_parietal)
# Save variables
train_R_parietal_tuber_burden = tuber_burden_R_parietal
# Calculate statistics
np.mean(tuber_burden_R_parietal), np.std(tuber_burden_R_parietal), np.median(tuber_burden_R_parietal), np.percentile(tuber_burden_R_parietal, 25), np.percentile(tuber_burden_R_parietal, 75)

(2899.2217, 3890.3457, 1505.0, 269.25, 3758.5)

#### Left occipital

In [47]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_occipital = []
for i in range(int(Y_train_L_occipital.shape[0]/64)):
    tuber_burden_single_MRI_L_occipital = np.sum(Y_train_L_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_occipital.append(tuber_burden_single_MRI_L_occipital)
# Save variables
train_L_occipital_tuber_burden = tuber_burden_L_occipital
# Calculate statistics
np.mean(tuber_burden_L_occipital), np.std(tuber_burden_L_occipital), np.median(tuber_burden_L_occipital), np.percentile(tuber_burden_L_occipital, 25), np.percentile(tuber_burden_L_occipital, 75)

(1789.7614, 2335.382, 1007.5, 3.5, 2501.0)

#### Right occipital

In [48]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_occipital = []
for i in range(int(Y_train_R_occipital.shape[0]/64)):
    tuber_burden_single_MRI_R_occipital = np.sum(Y_train_R_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_occipital.append(tuber_burden_single_MRI_R_occipital)
# Save variables
train_R_occipital_tuber_burden = tuber_burden_R_occipital
# Calculate statistics
np.mean(tuber_burden_R_occipital), np.std(tuber_burden_R_occipital), np.median(tuber_burden_R_occipital), np.percentile(tuber_burden_R_occipital, 25), np.percentile(tuber_burden_R_occipital, 75)

(1566.6875, 2190.4666, 917.5, 0.0, 1927.0)

## Validation set

#### Mask: gray matter

In [49]:
# Initialize the tensor where the reformatted masks will be saved
Y_gray_matter_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    gray_matter_mask_patches = patchify(gray_matter_mask, (64, 64, 64), step=64)
    gray_matter_mask_patches = np.reshape(gray_matter_mask_patches, (-1, gray_matter_mask_patches.shape[3], gray_matter_mask_patches.shape[4], gray_matter_mask_patches.shape[5]))
    Y_gray_matter_mask[i*64:i*64+64, :, :, :, 0] = gray_matter_mask_patches    

Y_gray_matter_mask = Y_gray_matter_mask.astype(np.float32)

In [50]:
# Mask the Y_validation
Y_validation_gray_matter = Y_validation * Y_gray_matter_mask

#### Mask: left frontal

In [51]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_frontal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_frontal_mask_patches = patchify(L_frontal_mask, (64, 64, 64), step=64)
    L_frontal_mask_patches = np.reshape(L_frontal_mask_patches, (-1, L_frontal_mask_patches.shape[3], L_frontal_mask_patches.shape[4], L_frontal_mask_patches.shape[5]))
    Y_L_frontal_mask[i*64:i*64+64, :, :, :, 0] = L_frontal_mask_patches    

Y_L_frontal_mask = Y_L_frontal_mask.astype(np.float32)

In [52]:
# Mask the Y_validation and Y_predictions
Y_validation_L_frontal = Y_validation * Y_L_frontal_mask

#### Mask: right frontal

In [53]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_frontal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_frontal_mask_patches = patchify(R_frontal_mask, (64, 64, 64), step=64)
    R_frontal_mask_patches = np.reshape(R_frontal_mask_patches, (-1, R_frontal_mask_patches.shape[3], R_frontal_mask_patches.shape[4], R_frontal_mask_patches.shape[5]))
    Y_R_frontal_mask[i*64:i*64+64, :, :, :, 0] = R_frontal_mask_patches    

Y_R_frontal_mask = Y_R_frontal_mask.astype(np.float32)

In [54]:
# Mask the Y_validation and Y_predictions
Y_validation_R_frontal = Y_validation * Y_R_frontal_mask

#### Mask: left temporal

In [55]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_temporal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_temporal_mask_patches = patchify(L_temporal_mask, (64, 64, 64), step=64)
    L_temporal_mask_patches = np.reshape(L_temporal_mask_patches, (-1, L_temporal_mask_patches.shape[3], L_temporal_mask_patches.shape[4], L_temporal_mask_patches.shape[5]))
    Y_L_temporal_mask[i*64:i*64+64, :, :, :, 0] = L_temporal_mask_patches    

Y_L_temporal_mask = Y_L_temporal_mask.astype(np.float32)

In [56]:
# Mask the Y_validation and Y_predictions
Y_validation_L_temporal = Y_validation * Y_L_temporal_mask

#### Mask: right temporal

In [57]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_temporal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_temporal_mask_patches = patchify(R_temporal_mask, (64, 64, 64), step=64)
    R_temporal_mask_patches = np.reshape(R_temporal_mask_patches, (-1, R_temporal_mask_patches.shape[3], R_temporal_mask_patches.shape[4], R_temporal_mask_patches.shape[5]))
    Y_R_temporal_mask[i*64:i*64+64, :, :, :, 0] = R_temporal_mask_patches    

Y_R_temporal_mask = Y_R_temporal_mask.astype(np.float32)

In [58]:
# Mask the Y_validation and Y_predictions
Y_validation_R_temporal = Y_validation * Y_R_temporal_mask

#### Mask: left parietal

In [59]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_parietal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_parietal_mask_patches = patchify(L_parietal_mask, (64, 64, 64), step=64)
    L_parietal_mask_patches = np.reshape(L_parietal_mask_patches, (-1, L_parietal_mask_patches.shape[3], L_parietal_mask_patches.shape[4], L_parietal_mask_patches.shape[5]))
    Y_L_parietal_mask[i*64:i*64+64, :, :, :, 0] = L_parietal_mask_patches    

Y_L_parietal_mask = Y_L_parietal_mask.astype(np.float32)

In [60]:
# Mask the Y_validation and Y_predictions
Y_validation_L_parietal = Y_validation * Y_L_parietal_mask

#### Mask: right parietal

In [61]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_parietal_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_parietal_mask_patches = patchify(R_parietal_mask, (64, 64, 64), step=64)
    R_parietal_mask_patches = np.reshape(R_parietal_mask_patches, (-1, R_parietal_mask_patches.shape[3], R_parietal_mask_patches.shape[4], R_parietal_mask_patches.shape[5]))
    Y_R_parietal_mask[i*64:i*64+64, :, :, :, 0] = R_parietal_mask_patches    

Y_R_parietal_mask = Y_R_parietal_mask.astype(np.float32)

In [62]:
# Mask the Y_validation and Y_predictions
Y_validation_R_parietal = Y_validation * Y_R_parietal_mask

#### Mask: left occipital

In [63]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_occipital_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_occipital_mask_patches = patchify(L_occipital_mask, (64, 64, 64), step=64)
    L_occipital_mask_patches = np.reshape(L_occipital_mask_patches, (-1, L_occipital_mask_patches.shape[3], L_occipital_mask_patches.shape[4], L_occipital_mask_patches.shape[5]))
    Y_L_occipital_mask[i*64:i*64+64, :, :, :, 0] = L_occipital_mask_patches    

Y_L_occipital_mask = Y_L_occipital_mask.astype(np.float32)

In [64]:
# Mask the Y_validation and Y_predictions
Y_validation_L_occipital = Y_validation * Y_L_occipital_mask

#### Mask: right occipital

In [65]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_occipital_mask = np.zeros( (n_all_validation, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_validation and Y_validation tensors
for i in range(len(included_validation_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_occipital_mask_patches = patchify(R_occipital_mask, (64, 64, 64), step=64)
    R_occipital_mask_patches = np.reshape(R_occipital_mask_patches, (-1, R_occipital_mask_patches.shape[3], R_occipital_mask_patches.shape[4], R_occipital_mask_patches.shape[5]))
    Y_R_occipital_mask[i*64:i*64+64, :, :, :, 0] = R_occipital_mask_patches    

Y_R_occipital_mask = Y_R_occipital_mask.astype(np.float32)

In [66]:
# Mask the Y_validation and Y_predictions
Y_validation_R_occipital = Y_validation * Y_R_occipital_mask

## Tuber burden per parcellations

#### Whole brain

In [67]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden = []
for i in range(int(Y_validation.shape[0]/64)):
    tuber_burden_single_MRI = np.sum(Y_validation[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden.append(tuber_burden_single_MRI)
# Save variables
validation_whole_brain_tuber_burden = tuber_burden
# Calculate statistics
np.mean(tuber_burden), np.std(tuber_burden), np.median(tuber_burden), np.percentile(tuber_burden, 25), np.percentile(tuber_burden, 75)

(35135.36, 33552.8, 27726.0, 7920.0, 54211.5)

#### Gray matter

In [68]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_gray_matter = []
for i in range(int(Y_validation_gray_matter.shape[0]/64)):
    tuber_burden_single_MRI_gray_matter = np.sum(Y_validation_gray_matter[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_gray_matter.append(tuber_burden_single_MRI_gray_matter)
# Save variables
validation_gray_matter_tuber_burden = tuber_burden_gray_matter
# Calculate statistics
np.mean(tuber_burden_gray_matter), np.std(tuber_burden_gray_matter), np.median(tuber_burden_gray_matter), np.percentile(tuber_burden_gray_matter, 25), np.percentile(tuber_burden_gray_matter, 75)

(32794.51, 31490.03, 25657.0, 5391.0, 51128.5)

#### Left frontal

In [69]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_frontal = []
for i in range(int(Y_validation_L_frontal.shape[0]/64)):
    tuber_burden_single_MRI_L_frontal = np.sum(Y_validation_L_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_frontal.append(tuber_burden_single_MRI_L_frontal)
# Save variables
validation_L_frontal_tuber_burden = tuber_burden_L_frontal
# Calculate statistics
np.mean(tuber_burden_L_frontal), np.std(tuber_burden_L_frontal), np.median(tuber_burden_L_frontal), np.percentile(tuber_burden_L_frontal, 25), np.percentile(tuber_burden_L_frontal, 75)

(7799.641, 7985.2715, 5154.0, 915.0, 11743.0)

#### Right frontal

In [70]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_frontal = []
for i in range(int(Y_validation_R_frontal.shape[0]/64)):
    tuber_burden_single_MRI_R_frontal = np.sum(Y_validation_R_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_frontal.append(tuber_burden_single_MRI_R_frontal)
# Save variables
validation_R_frontal_tuber_burden = tuber_burden_R_frontal
# Calculate statistics
np.mean(tuber_burden_R_frontal), np.std(tuber_burden_R_frontal), np.median(tuber_burden_R_frontal), np.percentile(tuber_burden_R_frontal, 25), np.percentile(tuber_burden_R_frontal, 75)

(7465.641, 8963.735, 3588.0, 858.5, 9475.5)

#### Left temporal

In [71]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_temporal = []
for i in range(int(Y_validation_L_temporal.shape[0]/64)):
    tuber_burden_single_MRI_L_temporal = np.sum(Y_validation_L_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_temporal.append(tuber_burden_single_MRI_L_temporal)
# Save variables
validation_L_temporal_tuber_burden = tuber_burden_L_temporal
# Calculate statistics
np.mean(tuber_burden_L_temporal), np.std(tuber_burden_L_temporal), np.median(tuber_burden_L_temporal), np.percentile(tuber_burden_L_temporal, 25), np.percentile(tuber_burden_L_temporal, 75)

(2643.7437, 3730.3655, 662.0, 0.0, 4102.5)

#### Right temporal

In [72]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_temporal = []
for i in range(int(Y_validation_R_temporal.shape[0]/64)):
    tuber_burden_single_MRI_R_temporal = np.sum(Y_validation_R_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_temporal.append(tuber_burden_single_MRI_R_temporal)
# Save variables
validation_R_temporal_tuber_burden = tuber_burden_R_temporal
# Calculate statistics
np.mean(tuber_burden_R_temporal), np.std(tuber_burden_R_temporal), np.median(tuber_burden_R_temporal), np.percentile(tuber_burden_R_temporal, 25), np.percentile(tuber_burden_R_temporal, 75)

(3632.4614, 5691.6426, 892.0, 0.0, 4476.0)

#### Left parietal

In [73]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_parietal = []
for i in range(int(Y_validation_L_parietal.shape[0]/64)):
    tuber_burden_single_MRI_L_parietal = np.sum(Y_validation_L_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_parietal.append(tuber_burden_single_MRI_L_parietal)
# Save variables
validation_L_parietal_tuber_burden = tuber_burden_L_parietal
# Calculate statistics
np.mean(tuber_burden_L_parietal), np.std(tuber_burden_L_parietal), np.median(tuber_burden_L_parietal), np.percentile(tuber_burden_L_parietal, 25), np.percentile(tuber_burden_L_parietal, 75)

(3351.1794, 4194.5654, 2050.0, 113.0, 5048.0)

#### Right parietal

In [74]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_parietal = []
for i in range(int(Y_validation_R_parietal.shape[0]/64)):
    tuber_burden_single_MRI_R_parietal = np.sum(Y_validation_R_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_parietal.append(tuber_burden_single_MRI_R_parietal)
# Save variables
validation_R_parietal_tuber_burden = tuber_burden_R_parietal
# Calculate statistics
np.mean(tuber_burden_R_parietal), np.std(tuber_burden_R_parietal), np.median(tuber_burden_R_parietal), np.percentile(tuber_burden_R_parietal, 25), np.percentile(tuber_burden_R_parietal, 75)

(4531.923, 4983.9424, 2299.0, 842.5, 7349.5)

#### Left occipital

In [75]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_occipital = []
for i in range(int(Y_validation_L_occipital.shape[0]/64)):
    tuber_burden_single_MRI_L_occipital = np.sum(Y_validation_L_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_occipital.append(tuber_burden_single_MRI_L_occipital)
# Save variables
validation_L_occipital_tuber_burden = tuber_burden_L_occipital
# Calculate statistics
np.mean(tuber_burden_L_occipital), np.std(tuber_burden_L_occipital), np.median(tuber_burden_L_occipital), np.percentile(tuber_burden_L_occipital, 25), np.percentile(tuber_burden_L_occipital, 75)

(1489.7693, 1664.0776, 930.0, 17.0, 2201.0)

#### Right occipital

In [76]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_occipital = []
for i in range(int(Y_validation_R_occipital.shape[0]/64)):
    tuber_burden_single_MRI_R_occipital = np.sum(Y_validation_R_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_occipital.append(tuber_burden_single_MRI_R_occipital)
# Save variables
validation_R_occipital_tuber_burden = tuber_burden_R_occipital
# Calculate statistics
np.mean(tuber_burden_R_occipital), np.std(tuber_burden_R_occipital), np.median(tuber_burden_R_occipital), np.percentile(tuber_burden_R_occipital, 25), np.percentile(tuber_burden_R_occipital, 75)

(2580.6924, 3092.3064, 1448.0, 72.0, 3729.5)

## Test set

#### Mask: gray matter

In [77]:
# Initialize the tensor where the reformatted masks will be saved
Y_gray_matter_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    gray_matter_mask_patches = patchify(gray_matter_mask, (64, 64, 64), step=64)
    gray_matter_mask_patches = np.reshape(gray_matter_mask_patches, (-1, gray_matter_mask_patches.shape[3], gray_matter_mask_patches.shape[4], gray_matter_mask_patches.shape[5]))
    Y_gray_matter_mask[i*64:i*64+64, :, :, :, 0] = gray_matter_mask_patches    

Y_gray_matter_mask = Y_gray_matter_mask.astype(np.float32)

In [78]:
# Mask the Y_test
Y_test_gray_matter = Y_test * Y_gray_matter_mask

#### Mask: left frontal

In [79]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_frontal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_frontal_mask_patches = patchify(L_frontal_mask, (64, 64, 64), step=64)
    L_frontal_mask_patches = np.reshape(L_frontal_mask_patches, (-1, L_frontal_mask_patches.shape[3], L_frontal_mask_patches.shape[4], L_frontal_mask_patches.shape[5]))
    Y_L_frontal_mask[i*64:i*64+64, :, :, :, 0] = L_frontal_mask_patches    

Y_L_frontal_mask = Y_L_frontal_mask.astype(np.float32)

In [80]:
# Mask the Y_test and Y_predictions
Y_test_L_frontal = Y_test * Y_L_frontal_mask

#### Mask: right frontal

In [81]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_frontal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_frontal_mask_patches = patchify(R_frontal_mask, (64, 64, 64), step=64)
    R_frontal_mask_patches = np.reshape(R_frontal_mask_patches, (-1, R_frontal_mask_patches.shape[3], R_frontal_mask_patches.shape[4], R_frontal_mask_patches.shape[5]))
    Y_R_frontal_mask[i*64:i*64+64, :, :, :, 0] = R_frontal_mask_patches    

Y_R_frontal_mask = Y_R_frontal_mask.astype(np.float32)

In [82]:
# Mask the Y_test and Y_predictions
Y_test_R_frontal = Y_test * Y_R_frontal_mask

#### Mask: left temporal

In [83]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_temporal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_temporal_mask_patches = patchify(L_temporal_mask, (64, 64, 64), step=64)
    L_temporal_mask_patches = np.reshape(L_temporal_mask_patches, (-1, L_temporal_mask_patches.shape[3], L_temporal_mask_patches.shape[4], L_temporal_mask_patches.shape[5]))
    Y_L_temporal_mask[i*64:i*64+64, :, :, :, 0] = L_temporal_mask_patches    

Y_L_temporal_mask = Y_L_temporal_mask.astype(np.float32)

In [84]:
# Mask the Y_test and Y_predictions
Y_test_L_temporal = Y_test * Y_L_temporal_mask

#### Mask: right temporal

In [85]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_temporal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_temporal_mask_patches = patchify(R_temporal_mask, (64, 64, 64), step=64)
    R_temporal_mask_patches = np.reshape(R_temporal_mask_patches, (-1, R_temporal_mask_patches.shape[3], R_temporal_mask_patches.shape[4], R_temporal_mask_patches.shape[5]))
    Y_R_temporal_mask[i*64:i*64+64, :, :, :, 0] = R_temporal_mask_patches    

Y_R_temporal_mask = Y_R_temporal_mask.astype(np.float32)

In [86]:
# Mask the Y_test and Y_predictions
Y_test_R_temporal = Y_test * Y_R_temporal_mask

#### Mask: left parietal

In [87]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_parietal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_parietal_mask_patches = patchify(L_parietal_mask, (64, 64, 64), step=64)
    L_parietal_mask_patches = np.reshape(L_parietal_mask_patches, (-1, L_parietal_mask_patches.shape[3], L_parietal_mask_patches.shape[4], L_parietal_mask_patches.shape[5]))
    Y_L_parietal_mask[i*64:i*64+64, :, :, :, 0] = L_parietal_mask_patches    

Y_L_parietal_mask = Y_L_parietal_mask.astype(np.float32)

In [88]:
# Mask the Y_test and Y_predictions
Y_test_L_parietal = Y_test * Y_L_parietal_mask

#### Mask: right parietal

In [89]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_parietal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_parietal_mask_patches = patchify(R_parietal_mask, (64, 64, 64), step=64)
    R_parietal_mask_patches = np.reshape(R_parietal_mask_patches, (-1, R_parietal_mask_patches.shape[3], R_parietal_mask_patches.shape[4], R_parietal_mask_patches.shape[5]))
    Y_R_parietal_mask[i*64:i*64+64, :, :, :, 0] = R_parietal_mask_patches    

Y_R_parietal_mask = Y_R_parietal_mask.astype(np.float32)

In [90]:
# Mask the Y_test and Y_predictions
Y_test_R_parietal = Y_test * Y_R_parietal_mask

#### Mask: left occipital

In [91]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_occipital_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_occipital_mask_patches = patchify(L_occipital_mask, (64, 64, 64), step=64)
    L_occipital_mask_patches = np.reshape(L_occipital_mask_patches, (-1, L_occipital_mask_patches.shape[3], L_occipital_mask_patches.shape[4], L_occipital_mask_patches.shape[5]))
    Y_L_occipital_mask[i*64:i*64+64, :, :, :, 0] = L_occipital_mask_patches    

Y_L_occipital_mask = Y_L_occipital_mask.astype(np.float32)

In [92]:
# Mask the Y_test and Y_predictions
Y_test_L_occipital = Y_test * Y_L_occipital_mask

#### Mask: right occipital

In [93]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_occipital_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_occipital_mask_patches = patchify(R_occipital_mask, (64, 64, 64), step=64)
    R_occipital_mask_patches = np.reshape(R_occipital_mask_patches, (-1, R_occipital_mask_patches.shape[3], R_occipital_mask_patches.shape[4], R_occipital_mask_patches.shape[5]))
    Y_R_occipital_mask[i*64:i*64+64, :, :, :, 0] = R_occipital_mask_patches    

Y_R_occipital_mask = Y_R_occipital_mask.astype(np.float32)

In [94]:
# Mask the Y_test and Y_predictions
Y_test_R_occipital = Y_test * Y_R_occipital_mask

## Tuber burden per parcellations

#### Whole brain

In [95]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden = []
for i in range(int(Y_test.shape[0]/64)):
    tuber_burden_single_MRI = np.sum(Y_test[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden.append(tuber_burden_single_MRI)
# Save variables
test_whole_brain_tuber_burden = tuber_burden
# Calculate statistics
np.mean(tuber_burden), np.std(tuber_burden), np.median(tuber_burden), np.percentile(tuber_burden, 25), np.percentile(tuber_burden, 75)

(64401.043, 61610.773, 50211.0, 14497.0, 100091.75)

#### Gray matter

In [96]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_gray_matter = []
for i in range(int(Y_test_gray_matter.shape[0]/64)):
    tuber_burden_single_MRI_gray_matter = np.sum(Y_test_gray_matter[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_gray_matter.append(tuber_burden_single_MRI_gray_matter)
# Save variables
test_gray_matter_tuber_burden = tuber_burden_gray_matter
# Calculate statistics
np.mean(tuber_burden_gray_matter), np.std(tuber_burden_gray_matter), np.median(tuber_burden_gray_matter), np.percentile(tuber_burden_gray_matter, 25), np.percentile(tuber_burden_gray_matter, 75)

(60669.645, 58252.367, 47947.0, 13180.75, 94486.5)

#### Left frontal

In [97]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_frontal = []
for i in range(int(Y_test_L_frontal.shape[0]/64)):
    tuber_burden_single_MRI_L_frontal = np.sum(Y_test_L_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_frontal.append(tuber_burden_single_MRI_L_frontal)
# Save variables
test_L_frontal_tuber_burden = tuber_burden_L_frontal
# Calculate statistics
np.mean(tuber_burden_L_frontal), np.std(tuber_burden_L_frontal), np.median(tuber_burden_L_frontal), np.percentile(tuber_burden_L_frontal, 25), np.percentile(tuber_burden_L_frontal, 75)

(13197.542, 14897.899, 8170.0, 1893.25, 18371.25)

#### Right frontal

In [98]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_frontal = []
for i in range(int(Y_test_R_frontal.shape[0]/64)):
    tuber_burden_single_MRI_R_frontal = np.sum(Y_test_R_frontal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_frontal.append(tuber_burden_single_MRI_R_frontal)
# Save variables
test_R_frontal_tuber_burden = tuber_burden_R_frontal
# Calculate statistics
np.mean(tuber_burden_R_frontal), np.std(tuber_burden_R_frontal), np.median(tuber_burden_R_frontal), np.percentile(tuber_burden_R_frontal, 25), np.percentile(tuber_burden_R_frontal, 75)

(13721.292, 12587.029, 9348.0, 4017.25, 19893.0)

#### Left temporal

In [99]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_temporal = []
for i in range(int(Y_test_L_temporal.shape[0]/64)):
    tuber_burden_single_MRI_L_temporal = np.sum(Y_test_L_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_temporal.append(tuber_burden_single_MRI_L_temporal)
# Save variables
test_L_temporal_tuber_burden = tuber_burden_L_temporal
# Calculate statistics
np.mean(tuber_burden_L_temporal), np.std(tuber_burden_L_temporal), np.median(tuber_burden_L_temporal), np.percentile(tuber_burden_L_temporal, 25), np.percentile(tuber_burden_L_temporal, 75)

(6587.771, 10051.172, 2301.0, 309.75, 9882.25)

#### Right temporal

In [100]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_temporal = []
for i in range(int(Y_test_R_temporal.shape[0]/64)):
    tuber_burden_single_MRI_R_temporal = np.sum(Y_test_R_temporal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_temporal.append(tuber_burden_single_MRI_R_temporal)
# Save variables
test_R_temporal_tuber_burden = tuber_burden_R_temporal
# Calculate statistics
np.mean(tuber_burden_R_temporal), np.std(tuber_burden_R_temporal), np.median(tuber_burden_R_temporal), np.percentile(tuber_burden_R_temporal, 25), np.percentile(tuber_burden_R_temporal, 75)

(6321.625, 9438.518, 2063.0, 73.0, 10631.5)

#### Left parietal

In [101]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_parietal = []
for i in range(int(Y_test_L_parietal.shape[0]/64)):
    tuber_burden_single_MRI_L_parietal = np.sum(Y_test_L_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_parietal.append(tuber_burden_single_MRI_L_parietal)
# Save variables
test_L_parietal_tuber_burden = tuber_burden_L_parietal
# Calculate statistics
np.mean(tuber_burden_L_parietal), np.std(tuber_burden_L_parietal), np.median(tuber_burden_L_parietal), np.percentile(tuber_burden_L_parietal, 25), np.percentile(tuber_burden_L_parietal, 75)

(6603.229, 7748.9404, 5266.0, 613.5, 8658.25)

#### Right parietal

In [102]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_parietal = []
for i in range(int(Y_test_R_parietal.shape[0]/64)):
    tuber_burden_single_MRI_R_parietal = np.sum(Y_test_R_parietal[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_parietal.append(tuber_burden_single_MRI_R_parietal)
# Save variables
test_R_parietal_tuber_burden = tuber_burden_R_parietal
# Calculate statistics
np.mean(tuber_burden_R_parietal), np.std(tuber_burden_R_parietal), np.median(tuber_burden_R_parietal), np.percentile(tuber_burden_R_parietal, 25), np.percentile(tuber_burden_R_parietal, 75)

(8716.167, 9539.631, 6706.0, 1034.25, 12782.0)

#### Left occipital

In [103]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_L_occipital = []
for i in range(int(Y_test_L_occipital.shape[0]/64)):
    tuber_burden_single_MRI_L_occipital = np.sum(Y_test_L_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_L_occipital.append(tuber_burden_single_MRI_L_occipital)
# Save variables
test_L_occipital_tuber_burden = tuber_burden_L_occipital
# Calculate statistics
np.mean(tuber_burden_L_occipital), np.std(tuber_burden_L_occipital), np.median(tuber_burden_L_occipital), np.percentile(tuber_burden_L_occipital, 25), np.percentile(tuber_burden_L_occipital, 75)

(3122.75, 3563.4158, 1772.0, 114.25, 5376.0)

#### Right occipital

In [104]:
# Calculate the tuber burden in each MRI (each 64 patches long)
tuber_burden_R_occipital = []
for i in range(int(Y_test_R_occipital.shape[0]/64)):
    tuber_burden_single_MRI_R_occipital = np.sum(Y_test_R_occipital[i*64:i*64+64, :, :, :, 0])
    # Collect the tuber burden from all MRIs
    tuber_burden_R_occipital.append(tuber_burden_single_MRI_R_occipital)
# Save variables
test_R_occipital_tuber_burden = tuber_burden_R_occipital
# Calculate statistics
np.mean(tuber_burden_R_occipital), np.std(tuber_burden_R_occipital), np.median(tuber_burden_R_occipital), np.percentile(tuber_burden_R_occipital, 25), np.percentile(tuber_burden_R_occipital, 75)

(3571.1042, 4743.506, 1407.5, 46.25, 5273.25)

## Comparison of the three groups with Kruskal-Wallis test

#### Whole brain

In [105]:
whole_brain_kruskal_wallis, whole_brain_p_value = kruskal(train_whole_brain_tuber_burden, validation_whole_brain_tuber_burden, test_whole_brain_tuber_burden)
whole_brain_kruskal_wallis, whole_brain_p_value

(16.80717048215236, 0.00022406256410298044)

#### Gray matter

In [106]:
gray_matter_kruskal_wallis, gray_matter_p_value = kruskal(train_gray_matter_tuber_burden, validation_gray_matter_tuber_burden, test_gray_matter_tuber_burden)
gray_matter_kruskal_wallis, gray_matter_p_value

(15.966078027336433, 0.0003412009310263858)

#### Left frontal

In [107]:
L_frontal_kruskal_wallis, L_frontal_p_value = kruskal(train_L_frontal_tuber_burden, validation_L_frontal_tuber_burden, test_L_frontal_tuber_burden)
L_frontal_kruskal_wallis, L_frontal_p_value

(11.663488470967271, 0.002932956748843751)

#### Right frontal

In [108]:
R_frontal_kruskal_wallis, R_frontal_p_value = kruskal(train_R_frontal_tuber_burden, validation_R_frontal_tuber_burden, test_R_frontal_tuber_burden)
R_frontal_kruskal_wallis, R_frontal_p_value

(19.769570133084084, 5.094392364520604e-05)

#### Left temporal

In [109]:
L_temporal_kruskal_wallis, L_temporal_p_value = kruskal(train_L_temporal_tuber_burden, validation_L_temporal_tuber_burden, test_L_temporal_tuber_burden)
L_temporal_kruskal_wallis, L_temporal_p_value

(11.978495732095098, 0.0025055478492856066)

#### Right temporal

In [110]:
R_temporal_kruskal_wallis, R_temporal_p_value = kruskal(train_R_temporal_tuber_burden, validation_R_temporal_tuber_burden, test_R_temporal_tuber_burden)
R_temporal_kruskal_wallis, R_temporal_p_value

(2.607222883612574, 0.2715493354351392)

#### Left parietal

In [111]:
L_parietal_kruskal_wallis, L_parietal_p_value = kruskal(train_L_parietal_tuber_burden, validation_L_parietal_tuber_burden, test_L_parietal_tuber_burden)
L_parietal_kruskal_wallis, L_parietal_p_value

(10.762247892312093, 0.004602645895583667)

#### Right parietal

In [112]:
R_parietal_kruskal_wallis, R_parietal_p_value = kruskal(train_R_parietal_tuber_burden, validation_R_parietal_tuber_burden, test_R_parietal_tuber_burden)
R_parietal_kruskal_wallis, R_parietal_p_value

(15.449336847584808, 0.0004417933037702567)

#### Left occipital

In [113]:
L_occipital_kruskal_wallis, L_occipital_p_value = kruskal(train_L_occipital_tuber_burden, validation_L_occipital_tuber_burden, test_L_occipital_tuber_burden)
L_occipital_kruskal_wallis, L_occipital_p_value

(4.3541515599341345, 0.11337257273296461)

#### Right occipital

In [114]:
R_occipital_kruskal_wallis, R_occipital_p_value = kruskal(train_R_occipital_tuber_burden, validation_R_occipital_tuber_burden, test_R_occipital_tuber_burden)
R_occipital_kruskal_wallis, R_occipital_p_value

(5.719927702052528, 0.05727083050979645)

## Adjustment for multiple comparisons considering also the p-values for sex, origin, number MRI per patient, age, and origin MRI from Table 1 calculated in the Demographics python script

In [115]:
other_p_values_from_Table_1 = [np.float64(0.26543962029543217),
 np.float64(0.0016050819555870177),
 np.float64(1.5809209070937485e-11),
 np.float64(4.253176932649638e-08),
 np.float64(0.2228992828118509)]

unadjusted_p_values = other_p_values_from_Table_1 + [whole_brain_p_value, gray_matter_p_value, L_frontal_p_value, R_frontal_p_value, L_temporal_p_value, R_temporal_p_value, L_parietal_p_value, R_parietal_p_value, L_occipital_p_value, R_occipital_p_value]

adjusted_p_values = false_discovery_control(unadjusted_p_values, method='bh')

unadjusted_p_values, adjusted_p_values

([0.26543962029543217,
  0.0016050819555870177,
  1.5809209070937485e-11,
  4.253176932649638e-08,
  0.2228992828118509,
  0.00022406256410298044,
  0.0003412009310263858,
  0.002932956748843751,
  5.094392364520604e-05,
  0.0025055478492856066,
  0.2715493354351392,
  0.004602645895583667,
  0.0004417933037702567,
  0.11337257273296461,
  0.05727083050979645],
 array([2.71549335e-01, 3.43946133e-03, 2.37138136e-10, 3.18988270e-07,
        2.57191480e-01, 8.40234615e-04, 1.02360279e-03, 4.88826125e-03,
        2.54719618e-04, 4.69790222e-03, 2.71549335e-01, 6.90396884e-03,
        1.10448326e-03, 1.41715716e-01, 7.80965871e-02]))

## Total (train, validation, and test set combined)

#### Whole brain

In [116]:
# Put together the three sets
total_tuber_burden = train_whole_brain_tuber_burden + validation_whole_brain_tuber_burden + test_whole_brain_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden), np.std(total_tuber_burden), np.median(total_tuber_burden), np.percentile(total_tuber_burden, 25), np.percentile(total_tuber_burden, 75)

(33887.33, 38567.652, 19702.0, 7264.0, 50103.0)

#### Gray matter

In [117]:
# Put together the three sets
total_tuber_burden_gray_matter = train_gray_matter_tuber_burden + validation_gray_matter_tuber_burden + test_gray_matter_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_gray_matter), np.std(total_tuber_burden_gray_matter), np.median(total_tuber_burden_gray_matter), np.percentile(total_tuber_burden_gray_matter, 25), np.percentile(total_tuber_burden_gray_matter, 75)

(31816.172, 36510.4, 18882.0, 5856.0, 47859.5)

#### Left frontal

In [118]:
# Put together the three sets
total_tuber_burden_L_frontal = train_L_frontal_tuber_burden + validation_L_frontal_tuber_burden + test_L_frontal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_L_frontal), np.std(total_tuber_burden_L_frontal), np.median(total_tuber_burden_L_frontal), np.percentile(total_tuber_burden_L_frontal, 25), np.percentile(total_tuber_burden_L_frontal, 75)

(7408.738, 9164.463, 4269.0, 1272.0, 10259.5)

#### Right frontal

In [119]:
# Put together the three sets
total_tuber_burden_R_frontal = train_R_frontal_tuber_burden + validation_R_frontal_tuber_burden + test_R_frontal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_R_frontal), np.std(total_tuber_burden_R_frontal), np.median(total_tuber_burden_R_frontal), np.percentile(total_tuber_burden_R_frontal, 25), np.percentile(total_tuber_burden_R_frontal, 75)

(7137.745, 8701.14, 3887.0, 1069.5, 9588.5)

#### Left temporal

In [120]:
# Put together the three sets
total_tuber_burden_L_temporal = train_L_temporal_tuber_burden + validation_L_temporal_tuber_burden + test_L_temporal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_L_temporal), np.std(total_tuber_burden_L_temporal), np.median(total_tuber_burden_L_temporal), np.percentile(total_tuber_burden_L_temporal, 25), np.percentile(total_tuber_burden_L_temporal, 75)

(2938.517, 5626.632, 803.0, 0.0, 3487.0)

#### Right temporal

In [121]:
# Put together the three sets
total_tuber_burden_R_temporal = train_R_temporal_tuber_burden + validation_R_temporal_tuber_burden + test_R_temporal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_R_temporal), np.std(total_tuber_burden_R_temporal), np.median(total_tuber_burden_R_temporal), np.percentile(total_tuber_burden_R_temporal, 25), np.percentile(total_tuber_burden_R_temporal, 75)

(3280.9658, 5491.9517, 1185.0, 36.5, 4040.0)

#### Left parietal

In [122]:
# Put together the three sets
total_tuber_burden_L_parietal = train_L_parietal_tuber_burden + validation_L_parietal_tuber_burden + test_L_parietal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_L_parietal), np.std(total_tuber_burden_L_parietal), np.median(total_tuber_burden_L_parietal), np.percentile(total_tuber_burden_L_parietal, 25), np.percentile(total_tuber_burden_L_parietal, 75)

(3423.289, 4750.636, 1414.0, 199.5, 5212.5)

#### Right parietal

In [123]:
# Put together the three sets
total_tuber_burden_R_parietal = train_R_parietal_tuber_burden + validation_R_parietal_tuber_burden + test_R_parietal_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_R_parietal), np.std(total_tuber_burden_R_parietal), np.median(total_tuber_burden_R_parietal), np.percentile(total_tuber_burden_R_parietal, 25), np.percentile(total_tuber_burden_R_parietal, 75)

(4202.981, 5940.701, 1866.0, 363.0, 5528.5)

#### Left occipital

In [124]:
# Put together the three sets
total_tuber_burden_L_occipital = train_L_occipital_tuber_burden + validation_L_occipital_tuber_burden + test_L_occipital_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_L_occipital), np.std(total_tuber_burden_L_occipital), np.median(total_tuber_burden_L_occipital), np.percentile(total_tuber_burden_L_occipital, 25), np.percentile(total_tuber_burden_L_occipital, 75)

(1988.559, 2583.8108, 1074.0, 7.0, 2776.5)

#### Right occipital

In [125]:
# Put together the three sets
total_tuber_burden_R_occipital = train_R_occipital_tuber_burden + validation_R_occipital_tuber_burden + test_R_occipital_tuber_burden
# Calculate statistics
np.mean(total_tuber_burden_R_occipital), np.std(total_tuber_burden_R_occipital), np.median(total_tuber_burden_R_occipital), np.percentile(total_tuber_burden_R_occipital, 25), np.percentile(total_tuber_burden_R_occipital, 75)

(2082.8784, 3058.565, 1038.0, 1.5, 2663.0)